In [1]:
using BenchmarkTools, Compat, DataFrames, Distributions, ForwardDiff

In [2]:
df = readtable("data/model_australia.txt", separator = ' ', header = false)

names!(df, [Symbol("x$i") for i in 1:4])

head(df)

,x1,x2,x3,x4
1,1,0,0,0
2,0,1,0,0
3,0,0,1,0
4,35,0,0,0
5,69,34,35,0
6,70,71,70,30


In [3]:
mixed_logit = DataFrame(P = 1.0:210.0)

names!(mixed_logit, [Symbol("Score")])

rand_contdist(Dist::Distribution) = quantile(Dist, rand())

rand_contdist (generic function with 1 method)

In [4]:
function simulate()
    for i = 1:210
        mixed_logit[i, 1] = rand_contdist(Uniform())
    end
end

function individual(θ::Vector, i::Int64)
    data = convert( Array, df[i*7-6:i*7-1, :])
    choices = convert(Array, df[i*7:i*7, :])
    alternatives = find(choices .== 0)
    choice = find(choices .== 1)[1]
    
    function utility(β::Vector, i::Int64)
        return dot(vec(data[:, i]), β)
    end
    
    function construct(γ::Vector, θ::Vector, β::Float64 = 0.0)
        for k = 1:length(γ)
            β += θ[k*1]+θ[k*2]*γ[k]
        end
        return β
    end
    
    function probability(θ::Vector, t::Float64 = 0.0)
        γ = mixed_logit[i, 1]
        β = θ[1:4]
        push!(β, construct([γ], θ[5:6]))
        push!(β, θ[length(θ)])        
        c = utility(β, choice)
        for alternative in alternatives
            t += exp(utility(β, alternative)-c)
        end
        return 1/(1+t)
    end
    
    return probability
end

simulate()

head(mixed_logit)

,Score
1,0.04256840174676979
2,0.41914150240927395
3,0.2819021590998132
4,0.6094246414581495
5,0.31147695573194256
6,0.8852635413755585


In [5]:
function f(θ::Vector, model::Float64 = 0.0, n::Int64 = 210)
    i = 1
    while i <= n
        probability = individual(θ, i)
        model += log(probability(θ))
        i += 1
    end
    return -model/n
end

f (generic function with 3 methods)

In [6]:
function g(x::Vector, n::Int64 = 210)
    t = zeros(length(x))
    for i = 1:n
        probability = individual(x, i)
        t += (1/probability(x))*ForwardDiff.gradient(probability, x)
    end
    return -t/n
end

function g!(x::Vector, storage::Vector)
    s = g(x)
    storage[1:length(s)] = s[1:length(s)]
end

g! (generic function with 1 method)

In [7]:
function H(x::Vector)
    return ForwardDiff.hessian(f, x)
end

function H!(x::Vector, storage::Matrix)
    s = H(x)
    n, m = size(s)
    storage[1:n, 1:m] = s[1:length(s)]
end

H! (generic function with 1 method)

In [8]:
function BHHH(x::Vector, n::Int64 = 210)
    t = zeros(length(x), length(x))
    for i = 1:n
        probability = individual(x, i)
        g = ForwardDiff.gradient(probability, x)
        t += g*(g')
    end
    return t/n
end

function BHHH!(x::Vector, storage::Matrix)
    s = BHHH(x)
    n, m = size(s)
    storage[1:n, 1:m] = s[1:length(s)]
end

BHHH! (generic function with 1 method)

In [9]:
function BFGS(B::Matrix, y::Vector, s::Vector)
    Bs = B*s
    return B-(Bs*Bs')/dot(s, Bs)+(y*y')/dot(s, y)
end

function BFGS!(B::Matrix, y::Vector, s::Vector)
    n, m = size(B)
    B[1:n, 1:m] = BFGS(B, y, s)
end

BFGS! (generic function with 1 method)

In [10]:
function SR1(B::Matrix, y::Vector, s::Vector)
    Bs = B*s
    return B+((y-Bs)*(y-Bs)')/((y-Bs)'*s)
end

function SR1!(B::Matrix, y::Vector, s::Vector)
    n, m = size(B)
    B[1:n, 1:m] = SR1(B, y, s)
end

SR1! (generic function with 1 method)

In [11]:
function newton(f::Function, g::Function, h::Function, x0::Vector,
        δ::Float64 = 1e-6, nmax::Int64 = 1000)
    k = 1
    x = x0
    n = length(x)
    δ *=δ
    H = eye(n)
    dfx = ones(n)
    g(x, dfx)
    while dot(dfx, dfx) > δ && k <= nmax
        k += 1
        g(x, dfx)
        h(x, H)
        x -= H\dfx
    end
    return x, k
end

newton (generic function with 3 methods)

In [12]:
println(newton(f, g!, H!, zeros(7)))

([5.19432, 3.87898, 3.17652, 0.0135687, -0.10293, 0.0132055, -0.0156356], 7)
